In [36]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

DIR_PATH = '../data/suicide/'

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# /kaggle/input
for dirname, _, filenames in os.walk(DIR_PATH):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

../data/suicide/HDI.csv
../data/suicide/master.csv
../data/suicide/postUSSR_90_00.csv
../data/suicide/postUSSR_suicide.csv


In [37]:
df = pd.read_csv(os.path.join(DIR_PATH, 'postUSSR_90_00.csv'), index_col=0)
df.head()

,country,year,sex,age,suicides_no,population,country-year,HDI,generation
0,Armenia,1990,male,35-54 years,25,332700,Armenia1990,0.632,Silent
1,Armenia,1990,male,75+ years,2,26900,Armenia1990,0.632,G.I. Generation
2,Armenia,1990,male,55-74 years,11,183800,Armenia1990,0.632,G.I. Generation
3,Armenia,1990,male,25-34 years,16,307100,Armenia1990,0.632,Boomers
4,Armenia,1990,female,35-54 years,15,358000,Armenia1990,0.632,Silent


In [38]:
corr = df.corr()
corr.style.background_gradient(cmap='coolwarm')

,year,suicides_no,population,HDI
year,1.000000,0.028082,0.004454,0.050529
suicides_no,0.028082,1.000000,0.686643,0.509122
population,0.004454,0.686643,1.000000,0.724138
HDI,0.050529,0.509122,0.724138,1.000000


Датасет уже после обработки в здесь - https://www.kaggle.com/firesworder/suicide-staff-eda

# Общие функции
Мб кстати пайплайн можно настроить. Я что то подобное делал уже

In [70]:
from sklearn.model_selection import train_test_split

def get_prepared_dataset(base_df, columns, target):
    base_df = base_df.copy()
    # выделение таргета и X
    y = base_df[target]
    X = base_df.loc[:, columns]
    
    return train_test_split(X, y, random_state=0)

train_X, val_X, train_y, val_y = get_prepared_dataset(df, ['HDI', 'country'], 'suicides_no')
print(train_X.shape, val_X.shape, train_y.shape, val_y.shape)

(989, 2) (330, 2) (989,) (330,)


In [104]:
def get_object_cols(base_df):
    object_cols = base_df.dtypes == 'object'
    object_cols = list(object_cols[object_cols].index)
    return object_cols

columns = get_object_cols(df.loc[:, ['HDI', 'country']])
columns

['country']

In [116]:
from sklearn.preprocessing import OneHotEncoder

def get_OH_encoder(base_df, object_cols):
    OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
    OH_encoder.fit(df[object_cols])
    return OH_encoder

sub_df = df.loc[:, ['HDI', 'country']]
object_columns = get_object_cols(sub_df)
OH_encoder = get_OH_encoder(sub_df, object_columns)
OH_encoder

OneHotEncoder(handle_unknown='ignore', sparse=False)

In [113]:
def get_OH_encodings(OH_encoder, base_df, object_cols):
    OH_cols = pd.DataFrame(OH_encoder.transform(base_df[object_cols]))
    OH_cols.index = base_df.index # восстанавливаю индексы после энкодинга
    OH_cols.columns = OH_encoder.get_feature_names_out() # восстанавливаю наименования
    df_num_cols = base_df.drop(object_cols, axis=1)
    encoded_df = pd.concat([df_num_cols, OH_cols], axis=1)
    return encoded_df

sub_df = df.loc[:, ['HDI', 'country']]
object_cols = get_object_cols(sub_df)
OH_encoder = get_OH_encoder(sub_df, object_columns)
train_X, val_X, _, _ = get_prepared_dataset(df, ['HDI', 'country'], 'suicides_no')
OH_X_train, OH_X_valid = get_OH_encodings(OH_encoder, train_X, object_cols), get_OH_encodings(OH_encoder, val_X, object_cols)
print(OH_X_train.shape, OH_X_valid.shape)

(989, 11) (330, 11)


In [75]:
from sklearn.metrics import mean_absolute_error

# Для оценки МАЕ построенной лин. модели(с возможностью каста полей таргета, перед оценкой)
# TODO: заменить train_X ит.п. на словарь
def get_MAE_score(model, train_X, val_X, train_y, val_y, mae_supp_func=None):
    pred_train = model.predict(train_X)
    pred_val = model.predict(val_X)
    
    if mae_supp_func:
        train_y, val_y = mae_supp_func(train_y), mae_supp_func(val_y)
        pred_train, pred_val = mae_supp_func(pred_train), mae_supp_func(pred_val)
        
    train_MAE = mean_absolute_error(train_y, pred_train)
    val_MAE = mean_absolute_error(val_y, pred_val)
    return train_MAE, val_MAE

In [128]:
from sklearn.linear_model import LinearRegression

def get_model(base_df, columns, target, mae_supp_func=None):
    train_X, val_X, train_y, val_y = get_prepared_dataset(base_df, columns, target)
    
    object_cols = get_object_cols(train_X)
    OH_encoder = None
    if object_cols:
        OH_encoder = get_OH_encoder(base_df.loc[:, columns], object_cols)
        OH_X_train = get_OH_encodings(OH_encoder, train_X, object_cols)
        OH_X_valid = get_OH_encodings(OH_encoder, val_X, object_cols)
    else:
        OH_X_train, OH_X_valid = train_X, val_X
       
    model = LinearRegression() 
    model.fit(OH_X_train, train_y)
    
    train_MAE, val_MAE = get_MAE_score(model, OH_X_train, OH_X_valid, train_y, val_y, mae_supp_func)
    
    return model, OH_encoder, {'train': train_MAE, 'valid': val_MAE}


get_model(df, ['HDI', 'country'], 'suicides_no')

(LinearRegression(),
 OneHotEncoder(handle_unknown='ignore', sparse=False),
 {'train': 628.9398382204247, 'valid': 464.24545454545455})

In [130]:
def get_column_comb_rating(base_df, target, mae_supp_func=None):
    # получение комбинаций(сочетаний без повторений) столбцов
    import itertools

    columns = np.array(base_df.columns)
    columns = columns[columns != target]

    columns_combinations = []
    for i in range(1, len(columns) + 1):
        columns_combinations += itertools.combinations(columns, i)
    
    # оценка каждого из сочетаний столбцов, по МАЕ
    df_column_option = []
    for option in columns_combinations:
        column_name = '|'.join(option)
        _, _, MAE = get_model(base_df, option, target, mae_supp_func)
        df_column_option.append([column_name, MAE['train'], MAE['valid']])

    df_column_option = pd.DataFrame(
        df_column_option, columns=['columns_name', 'train_MAE', 'val_MAE'])
    return df_column_option

get_column_comb_rating(df, 'suicides_no')

,columns_name,train_MAE,val_MAE
0,country,629.836198,464.403030
1,year,961.617972,875.040740
2,sex,938.162080,832.548248
3,age,918.685541,849.287879
4,population,668.941737,594.806073
...,...,...,...
250,country|year|sex|population|country-year|HDI|g...,775.889389,843.859426
251,country|year|age|population|country-year|HDI|g...,710.338326,772.770887
252,country|sex|age|population|country-year|HDI|ge...,775.661885,852.339984
253,year|sex|age|population|country-year|HDI|gener...,775.661885,852.339985


In [151]:
def get_predictions_df(model, model_cols, OH_encoder,  req_df):
    req_X = req_df.loc[:, model_cols]
    if OH_encoder:
        object_cols = get_object_cols(req_X)
        req_X = get_OH_encodings(OH_encoder, req_X, object_cols)
        
    preds = model.predict(req_X)
    preds = pd.Series(preds, index=req_df.index, name='predictions')
    req_df['preds'] = preds
    return req_df

model_cols, target = ['country', 'HDI'], 'suicides_no'
req_df = df.sample(n=10)
model, OH_encoder, _ = get_model(df, model_cols, target)
result_df = get_predictions_df(model, model_cols, OH_encoder, req_df)
result_df

,country,year,sex,age,suicides_no,population,country-year,HDI,generation,preds
677,Kyrgyzstan,1992,female,75+ years,7,56800,Kyrgyzstan1992,0.562,G.I. Generation,55.5
77,Armenia,1996,female,35-54 years,7,471200,Armenia1996,0.648,Boomers,-2.0
524,Kazakhstan,1990,female,35-54 years,225,1845400,Kazakhstan1990,0.690,Silent,361.0
322,Belarus,1994,male,5-14 years,16,833400,Belarus1994,0.683,Millenials,277.5
152,Azerbaijan,1991,female,75+ years,1,99300,Azerbaijan1991,0.609,G.I. Generation,14.0
139,Azerbaijan,1990,female,25-34 years,7,677800,Azerbaijan1990,0.609,Boomers,14.0
1254,Uzbekistan,1996,female,55-74 years,55,959900,Uzbekistan1996,0.594,Silent,125.0
307,Belarus,1993,female,35-54 years,164,1323600,Belarus1993,0.683,Boomers,277.5
930,Turkmenistan,1991,female,15-24 years,31,358300,Turkmenistan1991,0.666,Generation X,26.5
716,Kyrgyzstan,1995,female,35-54 years,35,394200,Kyrgyzstan1995,0.562,Boomers,55.5


# Модель лин. регрессии
## Прямой проход

In [154]:
df_column_option = get_column_comb_rating(df, 'suicides_no')
df_column_option.head()

,columns_name,train_MAE,val_MAE
0,country,629.836198,464.403030
1,year,961.617972,875.040740
2,sex,938.162080,832.548248
3,age,918.685541,849.287879
4,population,668.941737,594.806073


In [155]:
df_column_option.sort_values(by=['train_MAE', 'val_MAE']).head(5)

,columns_name,train_MAE,val_MAE
69,year|country-year|HDI,613.120324,493.366667
18,year|country-year,613.419816,493.617967
39,country|year|country-year,613.515672,493.648485
13,country|HDI,628.939838,464.245455
0,country,629.836198,464.403030


Лучший результат показали по величине ошибки показали country и country-year, а также комбинации с оными.

В целом, ошибка вышла очень большая и это немного странно, учитывая "отделенность" столбцов страны(one-hot).

In [165]:
columns = ['country']
model, OH_encoder, MAE = get_model(df, columns, target)
print(f'Train MAE: {MAE["train"]}, Val MAE: {MAE["valid"]}')
req_df = df.sample(n=10)
get_predictions_df(model, columns, OH_encoder, req_df)

Train MAE: 629.8361981799798, Val MAE: 464.4030303030303


,country,year,sex,age,suicides_no,population,country-year,HDI,generation,preds
407,Georgia,1990,male,5-14 years,0,444500,Georgia1990,0.672,Generation X,-8.0
803,Russian Federation,1991,female,5-14 years,65,11204500,Russian Federation1991,0.824,Millenials,4424.0
689,Kyrgyzstan,1993,male,15-24 years,73,414400,Kyrgyzstan1993,0.562,Generation X,40.0
772,Kyrgyzstan,2000,female,75+ years,10,56276,Kyrgyzstan2000,0.593,G.I. Generation,40.0
921,Turkmenistan,1990,female,25-34 years,16,306400,Turkmenistan1990,0.666,Boomers,8.0
1231,Uzbekistan,1994,female,15-24 years,110,2132800,Uzbekistan1994,0.594,Generation X,128.0
1091,Ukraine,1993,female,5-14 years,20,3738000,Ukraine1993,0.662,Millenials,1192.0
1288,Uzbekistan,1999,male,75+ years,13,98024,Uzbekistan1999,0.594,G.I. Generation,128.0
787,Russian Federation,1990,female,35-54 years,2467,19730400,Russian Federation1990,0.824,Silent,4424.0
839,Russian Federation,1994,female,5-14 years,75,11447200,Russian Federation1994,0.824,Millenials,4424.0


По Украине сильный промах, например, аж в 10 раз. И по некоторым другим странам тоже.

Если это по среднему - это отчасти объясняет промах.

In [48]:
df[df.country == 'Ukraine'].groupby('year').suicides_no.mean()

year
1990     887.083333
1991     891.666667
1992     972.166667
1993    1039.083333
1994    1152.166667
1995    1211.750000
1996    1263.333333
1997    1241.916667
1998    1232.416667
1999    1196.916667
2000    1205.583333
Name: suicides_no, dtype: float64

Предсказание 1176 - очень близко к средним по Украине, даже с разбивкой по годам.

Т.е. результат модели - правильный, если учитывать конкретно это поле.

In [49]:
df[df.country == 'Kyrgyzstan'].groupby('year').suicides_no.mean()

year
1990    45.583333
1991    45.250000
1992    40.916667
1993    46.916667
1994    48.416667
1995    50.416667
1996    40.000000
1997    41.333333
1998    42.416667
1999    46.333333
2000    42.500000
Name: suicides_no, dtype: float64

Кыргыстан тоже рассчитан моделью правильно: 46 - предсказание, ~46 - средние по годам.

## С нормализацией по нат. логарифму
Нормализую столбец населения и количества самоубийств, применив логарифм к кол-ву населения и самоубийств

In [166]:
log_with_zero = lambda x: np.log(x) if x > 0 else 0
cols_to_norm = ['population', 'suicides_no']
df_norm_ln = df.copy()
for col in cols_to_norm:
    df_norm_ln.loc[:, col] = df.loc[:, col].apply(log_with_zero)
df_norm_ln.tail(5)

,country,year,sex,age,suicides_no,population,country-year,HDI,generation
1315,Georgia,1993,male,5-14 years,0.000000,12.937599,Georgia1993,0.672,Millenials
1316,Georgia,1993,female,55-74 years,2.708050,13.149054,Georgia1993,0.672,Silent
1317,Georgia,1993,male,55-74 years,3.784190,12.839868,Georgia1993,0.672,Silent
1318,Georgia,1993,female,75+ years,1.791759,11.800733,Georgia1993,0.672,G.I. Generation
1319,Georgia,1993,male,75+ years,1.791759,11.008330,Georgia1993,0.672,G.I. Generation


In [173]:
df_column_option = get_column_comb_rating(df_norm_ln, 'suicides_no', mae_supp_func=np.exp)
df_column_option.sort_values(by=['train_MAE', 'val_MAE']).head(5)

,columns_name,train_MAE,val_MAE
212,sex|age|population|country-year|HDI,157.866765,100.334761
239,year|sex|age|population|country-year|HDI,158.145613,100.022258
233,country|sex|age|population|country-year|HDI,158.192534,100.199357
182,country|sex|age|population|country-year,158.251310,100.047313
197,year|sex|age|population|country-year,158.347134,100.046553


Очевидно, что появились новые кандидаты набора колонок и сами комбинации стали сильно полнее.

Восстановлю(из натурального логарифма) не длинную комбинацию **sex|age|population|country-year|HDI**

In [174]:
col_combination = 'sex|age|population|country-year|HDI'.split('|')
col_combination

['sex', 'age', 'population', 'country-year', 'HDI']

In [176]:
model, OH_encoder, MAE = get_model(df_norm_ln, col_combination, 'suicides_no', mae_supp_func=np.exp)
print(f'Train MAE: {MAE["train"]}, Val MAE: {MAE["valid"]}')
result = get_predictions_df(model, col_combination, OH_encoder, req_df=df_norm_ln.sample(10))
# восстанавливаю от логарифмирования по населению и кол-ву суицидов(ну и предсказания, для случая предск. суицидов)
result.loc[:, ['population', 'preds', 'suicides_no']] = \
    result.loc[:, ['population', 'preds', 'suicides_no']].apply(np.exp)
result.loc[:, 'preds'] = result.preds.round()
result

Train MAE: 157.8667647848423, Val MAE: 100.33476121703279


,country,year,sex,age,suicides_no,population,country-year,HDI,generation,preds
364,Belarus,1998,male,15-24 years,287.0,762700.0,Belarus1998,0.683,Generation X,213.0
731,Kyrgyzstan,1996,female,5-14 years,3.0,561600.0,Kyrgyzstan1996,0.593,Millenials,3.0
653,Kyrgyzstan,1990,female,55-74 years,38.0,242200.0,Kyrgyzstan1990,0.615,G.I. Generation,30.0
97,Armenia,1998,male,55-74 years,16.0,243300.0,Armenia1998,0.648,Silent,14.0
378,Belarus,1999,female,55-74 years,170.0,1157900.0,Belarus1999,0.683,Silent,263.0
1199,Uzbekistan,1991,female,5-14 years,7.0,2566500.0,Uzbekistan1991,0.594,Millenials,10.0
341,Belarus,1996,female,75+ years,57.0,302400.0,Belarus1996,0.683,G.I. Generation,91.0
720,Kyrgyzstan,1996,male,55-74 years,81.0,194300.0,Kyrgyzstan1996,0.593,Silent,69.0
355,Belarus,1997,female,35-54 years,188.0,1415500.0,Belarus1997,0.683,Boomers,181.0
231,Azerbaijan,1998,male,25-34 years,12.0,683600.0,Azerbaijan1998,0.640,Generation X,12.0


Точность получилась весьма неплохая. Разница между реальным значением и предсказанием - не выглядит значительной(пусть даже числа МАЕ и показывают довольно большую ошибку)

## Нормализация мин-макс

In [177]:
def get_min_max_norm(series):
    s_min, s_max = series.min(), series.max()
    s_range = s_max - s_min
    return (series - s_min) / (s_range), s_range

cast, s_range = get_min_max_norm(df.suicides_no)
cast, s_range

(0       0.001119
 1       0.000090
 2       0.000492
 3       0.000716
 4       0.000672
           ...   
 1315    0.000000
 1316    0.000672
 1317    0.001970
 1318    0.000269
 1319    0.000269
 Name: suicides_no, Length: 1319, dtype: float64,
 22338)

In [178]:
# Нормализую столбец населения и количества самоубийств, приведя его от 0 до 1.(или логарифм)
# сначала логарифм
df_minmax = df.copy()
s_ranges = {}
for column in ['population', 'suicides_no']:
    df_minmax.loc[:, column], s_ranges[column] = get_min_max_norm(df_minmax[column])
df_minmax.tail(5)

,country,year,sex,age,suicides_no,population,country-year,HDI,generation
1315,Georgia,1993,male,5-14 years,0.000000,0.017543,Georgia1993,0.672,Millenials
1316,Georgia,1993,female,55-74 years,0.000672,0.021833,Georgia1993,0.672,Silent
1317,Georgia,1993,male,55-74 years,0.001970,0.015847,Georgia1993,0.672,Silent
1318,Georgia,1993,female,75+ years,0.000269,0.005170,Georgia1993,0.672,G.I. Generation
1319,Georgia,1993,male,75+ years,0.000269,0.001971,Georgia1993,0.672,G.I. Generation


In [184]:
mae_func_minmax = lambda x: x * s_ranges['suicides_no']
df_column_option = get_column_comb_rating(df_minmax, 'suicides_no', mae_supp_func=mae_func_minmax)
df_column_option.sort_values(by=['train_MAE', 'val_MAE']).head(5)

,columns_name,train_MAE,val_MAE
18,year|country-year,613.419816,493.617967
69,year|country-year|HDI,613.829810,494.125456
39,country|year|country-year,613.829956,493.809407
0,country,628.480444,463.570511
13,country|HDI,628.803901,464.121945


In [181]:
col_combination = 'year|country-year'.split('|')
col_combination

['year', 'country-year']

In [185]:
model, OH_encoder, MAE = get_model(df_minmax, col_combination, 'suicides_no', mae_func_minmax)
print(f'Train MAE: {MAE["train"]}, Val MAE: {MAE["valid"]}')
result = get_predictions_df(model, col_combination, OH_encoder, req_df=df_minmax.sample(10))

for column in ['population', 'preds', 'suicides_no']:
    s_range = s_ranges['suicides_no'] if column == 'preds' else s_ranges[column]
    result.loc[:, column] = round(result[column] * s_range)
result

Train MAE: 613.4198156634934, Val MAE: 493.6179671170541


,country,year,sex,age,suicides_no,population,country-year,HDI,generation,preds
945,Turkmenistan,1992,female,25-34 years,10.0,305000.0,Turkmenistan1992,0.666,Boomers,30.0
939,Turkmenistan,1992,female,75+ years,5.0,17300.0,Turkmenistan1992,0.666,G.I. Generation,30.0
207,Azerbaijan,1996,male,35-54 years,14.0,678400.0,Azerbaijan1996,0.640,Boomers,6.0
232,Azerbaijan,1998,female,55-74 years,4.0,476200.0,Azerbaijan1998,0.640,Silent,3.0
613,Kazakhstan,1998,male,35-54 years,1459.0,1709000.0,Kazakhstan1998,0.679,Boomers,489.0
1065,Ukraine,1991,female,15-24 years,137.0,3479600.0,Ukraine1991,0.662,Generation X,635.0
760,Kyrgyzstan,1999,female,75+ years,11.0,39300.0,Kyrgyzstan1999,0.593,G.I. Generation,46.0
632,Kazakhstan,1999,female,35-54 years,198.0,1894500.0,Kazakhstan1999,0.679,Boomers,393.0
594,Kazakhstan,1996,female,55-74 years,211.0,1157700.0,Kazakhstan1996,0.679,Silent,429.0
1133,Ukraine,1997,female,75+ years,409.0,1683700.0,Ukraine1997,0.668,G.I. Generation,1185.0


Ошибка с минмакс - оказалась примерной такой же, что и при отсутсвии нормализации.

Мб даже больше

## Нормализация отклонения от среднего

In [186]:
def norm_by_std(series):
    s_mean, s_std = series.mean(), series.std()
    series = (series - s_mean) / s_std if s_std != 0 else 0.
    return series, {'std': s_std, 'mean': s_mean}

norm_by_std(df.suicides_no)

(0      -0.279052
 1      -0.289607
 2      -0.285477
 3      -0.283182
 4      -0.283641
           ...   
 1315   -0.290525
 1316   -0.283641
 1317   -0.270332
 1318   -0.287771
 1319   -0.287771
 Name: suicides_no, Length: 1319, dtype: float64,
 {'std': 2179.0020519471304, 'mean': 633.054586808188})

In [196]:
def restore_norm_by_std(series, s_norm_data):
    series = (series * s_norm_data['std']) + s_norm_data['mean']
    return series

demo_series, demo_norm_data = norm_by_std(df.suicides_no)
restore_norm_by_std(demo_series, demo_norm_data) - df.suicides_no

0       0.0
1       0.0
2       0.0
3       0.0
4       0.0
       ... 
1315    0.0
1316    0.0
1317    0.0
1318    0.0
1319    0.0
Name: suicides_no, Length: 1319, dtype: float64

In [205]:
df_std = df.copy()
s_norm_data = {}
for column in ['population', 'suicides_no']:
    df_std.loc[:, column], s_norm_data[column] = norm_by_std(df_std[column])
df_std.tail(5)

,country,year,sex,age,suicides_no,population,country-year,HDI,generation
1315,Georgia,1993,male,5-14 years,-0.290525,-0.446026,Georgia1993,0.672,Millenials
1316,Georgia,1993,female,55-74 years,-0.283641,-0.420172,Georgia1993,0.672,Silent
1317,Georgia,1993,male,55-74 years,-0.270332,-0.456249,Georgia1993,0.672,Silent
1318,Georgia,1993,female,75+ years,-0.287771,-0.520598,Georgia1993,0.672,G.I. Generation
1319,Georgia,1993,male,75+ years,-0.287771,-0.539875,Georgia1993,0.672,G.I. Generation


In [206]:
mae_func_std = lambda x: restore_norm_by_std(x, s_norm_data['suicides_no'])

In [210]:
df_column_option = get_column_comb_rating(df_std, 'suicides_no')
df_column_option.sort_values(by=['train_MAE', 'val_MAE']).head(10)

,columns_name,train_MAE,val_MAE
18,year|country-year,0.281514,0.226534
39,country|year|country-year,0.281684,0.226631
69,year|country-year|HDI,0.281889,0.226931
13,country|HDI,0.288614,0.213088
0,country,0.290333,0.214861
54,country|country-year|HDI,0.291547,0.237142
143,year|population|country-year|HDI,0.297284,0.301381
30,population|country-year,0.297509,0.302227
66,year|population|country-year,0.297509,0.302227
178,country|year|population|country-year|HDI,0.297676,0.302831


In [ ]:
col_combination = 'year|country-year'.split('|')
col_combination

In [ ]:
model, OH_encoder, MAE = get_model(df_std, col_combination, 'suicides_no', mae_func_std)
print(f'Train MAE: {MAE["train"]}, Val MAE: {MAE["valid"]}')
result = get_predictions_df(model, col_combination, OH_encoder, req_df=df_std.sample(10))

for column in ['population', 'preds', 'suicides_no']:
    s_norm = s_norm_data['suicides_no'] if column == 'preds' else s_norm_data[column]
    result.loc[:, column] = round(restore_norm_by_std(result[column], s_norm))
result.head(10)

Результаты +- такие же, что и без нормировки.
_____________
Вообщем, вариант с нормализацией значений нат. логарифмом оказался самым сильным.